In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob

In [4]:
run = 'historical_run/'
os.chdir('/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/'+run)

print(os.getcwd()) 


/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/historical_run


In [5]:
output_paths = glob.glob('var_out/*')
input_paths = glob.glob('clm_out/*')

print(len(output_paths))
print(len(input_paths))

200
300


In [8]:
# cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
#             , names = output_col_names)# output_paths

In [9]:
# cohort_output.loc[0]

In [6]:

output_group = [re.findall('[0-9]+$', w) for w in output_paths]
output_group = [item for sublist in output_group for item in sublist]

output_var = [re.findall('AVLN|NCE|NPP|SOILORGC|VEGC|GPP|NEP|SMOIST|SOLN|VEGN', w) for w in output_paths]
output_var = [item for sublist in output_var for item in sublist]

input_group = [re.findall('[0-9]+$', w) for w in input_paths]
input_group = [item.upper() for sublist in input_group for item in sublist]

input_var = [re.findall('PREC|CLDS|PAR|TAIR|VPR|WS10|NIRR', w) for w in input_paths]
input_var = [item for sublist in input_var for item in sublist]


In [7]:
# print(len(input_paths))
# print((input_paths))
# print(len(input_group))
# print(len(input_var))
print(len(output_paths))
# print((output_paths))
print(len(output_group))
print(len(output_var))

200
200
200


In [15]:
# output_group

In [8]:
#make tables with the relevant read in info

input_data = pd.DataFrame(
{'input_paths': input_paths,
'input_group': input_group,
 'input_var': input_var
}
)

output_data = pd.DataFrame(
{'output_paths': output_paths,
'output_group': output_group,
 'output_var': output_var
}
)

In [9]:
input_data

,input_paths,input_group,input_var
0,clm_out/PAR.csv20048,20048,PAR
1,clm_out/PAR.csv10024,10024,PAR
2,clm_out/TAIR.csv23807,23807,TAIR
3,clm_out/PAR.csv53879,53879,PAR
4,clm_out/TAIR.csv27566,27566,TAIR
...,...,...,...
295,clm_out/TAIR.csv37590,37590,TAIR
296,clm_out/VPR.csv6265,6265,VPR
297,clm_out/VPR.csv11277,11277,VPR
298,clm_out/PREC.csv2506,2506,PREC


In [10]:

input_data = pd.pivot(input_data, index = 'input_group', columns = 'input_var', values = 'input_paths').reset_index()

read_in_data = input_data.merge(output_data, left_on = 'input_group', right_on = 'output_group')

In [11]:
read_in_data.to_csv('/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/'+run+'all_parallel_variable_info.csv', index=False)

In [33]:
read_in_data

,input_group,NIRR,PAR,PREC,TAIR,VPR,WS10,output_paths,output_group,output_var
0,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/SOILORGC.csv0,0,SOILORGC
1,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/NPP.csv0,0,NPP
2,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/VEGC.csv0,0,VEGC
3,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/GPP.csv0,0,GPP
4,10024,clm_out/NIRR.csv10024,clm_out/PAR.csv10024,clm_out/PREC.csv10024,clm_out/TAIR.csv10024,clm_out/VPR.csv10024,clm_out/WS10.csv10024,var_out/GPP.csv10024,10024,GPP
...,...,...,...,...,...,...,...,...,...,...
195,7518,clm_out/NIRR.csv7518,clm_out/PAR.csv7518,clm_out/PREC.csv7518,clm_out/TAIR.csv7518,clm_out/VPR.csv7518,clm_out/WS10.csv7518,var_out/NPP.csv7518,7518,NPP
196,8771,clm_out/NIRR.csv8771,clm_out/PAR.csv8771,clm_out/PREC.csv8771,clm_out/TAIR.csv8771,clm_out/VPR.csv8771,clm_out/WS10.csv8771,var_out/NPP.csv8771,8771,NPP
197,8771,clm_out/NIRR.csv8771,clm_out/PAR.csv8771,clm_out/PREC.csv8771,clm_out/TAIR.csv8771,clm_out/VPR.csv8771,clm_out/WS10.csv8771,var_out/VEGC.csv8771,8771,VEGC
198,8771,clm_out/NIRR.csv8771,clm_out/PAR.csv8771,clm_out/PREC.csv8771,clm_out/TAIR.csv8771,clm_out/VPR.csv8771,clm_out/WS10.csv8771,var_out/SOILORGC.csv8771,8771,SOILORGC
